In [1]:
%load_ext zipline

In [32]:
%%zipline --start 2016-1-1 --end 2018-1-25 -b futures -o quick.pickle

import pandas as pd
import numpy as np
import talib
from zipline.api import (order, record, symbol, continuous_future,
                         future_symbol, get_open_orders, order_target_percent,
                         set_slippage, set_commission, get_datetime)
from zipline.finance.slippage import FixedSlippage
from zipline.finance.commission import PerTrade
from contracts import contracts
import sys

FAST_MA = 50
SLOW_MA = 100
BREAKOUT = 50 # breakout beyond x days max/min
STOP = 2 # stop after x ATRs
RISK = .2 # % of capital daily risk per position


def initialize(context):
    set_slippage(us_futures=FixedSlippage(spread=0.0))
    set_commission(us_futures=PerTrade(0))
    context.contracts = [
        continuous_future(contract,
                          offset=0,
                          adjustment='mul',
                          roll='volume')
        for contract in contracts]

    context.min_max = {}
    context.COUNTER = 0

        

def handle_data(context, data):
    hist = data.history(context.contracts,
                        fields = ['price', 'high', 'low'],
                        bar_count = SLOW_MA + 10,
                        frequency = '1d')


    slow_ma = hist['price'].apply(lambda x: 
                                  talib.EMA(x.as_matrix(), 
                                            timeperiod=SLOW_MA)[-1])
    fast_ma = hist['price'].apply(lambda x: 
                                  talib.EMA(x.as_matrix(), 
                                            timeperiod=FAST_MA)[-1])
    atr = hist.apply(lambda x: talib.ATR(x['high'].fillna(x['price']).as_matrix(), 
                                         x['low'].fillna(x['price']).as_matrix(),
                                         x['price'].as_matrix(), 
                                         timeperiod=SLOW_MA)[-1], 
                     axis=(1,0))
    



    # breakout above is a buy signal
    upper = hist['price'][-BREAKOUT-1:-2].max(axis=0)
    # breakout below is a sell signal
    lower = hist['price'][-BREAKOUT-1:-2].min(axis=0)
    # last price
    price = hist['price'].iloc[-1,:]

    weights = RISK * price / (atr * 100)
    
    longs = ((price > upper) & (fast_ma > slow_ma)) * weights 
    shorts = ((price < lower) & (fast_ma < slow_ma)) * -weights   
 
    
    positions = longs + shorts
   
    
    positions.index = data.current(positions.index, 'contract')
    #atr.index = data.current(atr.index, 'contract')
    price.index = data.current(price.index, 'contract')
    #atr_dict = atr.to_dict()
    positions = positions[positions != 0]
    
    # rollover expiring contracts
    signals = pd.Series()
    for cont, pos in context.portfolio.positions.items():
        if cont not in price.index:
            weight = context.portfolio.current_portfolio_weights[cont]
            root = cont.root_symbol
            # close existing contract
            positions[cont] = 0
            signals[cont] = 0
            current = data.current(continuous_future(root), 'contract')
            # open current contract
            positions[current] = weight
            signals[current] = 1
            # still need the old contract for stop-loss calculation
            atr_dict[cont] = atr_dict[current]

    # implement stop-loss 
    stop_out = []        
    for contract, position in context.portfolio.positions.items():
        if contract in context.min_max:
            context.min_max[contract] = (min(context.min_max[contract][0],
                                             position.cost_basis,
                                             position.last_sale_price),
                                         max(context.min_max[contract][1],
                                             position.cost_basis,
                                             position.last_sale_price))
        else:
          context.min_max[contract] = (min(position.cost_basis,
                                           position.last_sale_price),
                                       max(position.cost_basis,
                                           position.last_sale_price))    

        # calculate stop loss level
        try:
            if position.amount > 0:
                stop_price = context.min_max[contract][1] - atr[contract] * STOP
                if price[contract] <= stop_price:
                    positions[contract] = 0
                    stop_out.append(contract)
                
            if position.amount < 0:
                stop_price = context.min_max[contract][0] + atr[contract] * STOP
                if price[contract] >= stop_price:
                    positions[contract] = 0
                    stop_out.append(contract)
        except (KeyError, IndexError):
            pass
            #print('key error in stop-loss function: ', contract)
            #print(context.min_max)
            #print(atr_dict)

    temp = []        
    for key in context.min_max.keys():
        if key not in context.portfolio.positions.keys():
            temp.append(key)
    for key in temp:
        del context.min_max[key]
            
    # list of positions not to be affected by stop-loss
    current_positions = list(context.portfolio.positions.keys())
    for pos in stop_out:
        if pos in current_positions:
            current_positions.remove(pos)


    # fillna is a temporary fix
    weights_dict = positions.fillna(0).to_dict()
    #print(weights_dict)
    #print('orders:')
    for asset, target in weights_dict.items():
        if target not in current_positions:
            print('TRADING')
            print(asset, target)
            order_target_percent(asset, target)
    

    #print('portfolio:')
    #print(context.portfolio)
    #print('positions:')
    #print(context.portfolio.positions)
    context.COUNTER += 1
    if context.COUNTER > 2:
        sys.exit()




Series([], dtype: float64)
TRADING
Future(16692 [SMH16]) -0.09432381798065942
TRADING
Future(16698 [BPH16]) -0.26403263895917617
TRADING
Future(16676 [KWH16]) -0.07720414179448136
TRADING
Future(16701 [WH16]) -0.07659558974532849
TRADING
Future(16684 [CH16]) -0.0958090839386867


KeyError: "None of [{Future(16692 [SMH16]): Position({'amount': -35, 'last_sale_price': 266.3, 'cost_basis': 266.3, 'last_sale_date': Timestamp('2016-01-05 21:00:00+0000', tz='UTC'), 'asset': Future(16692 [SMH16])}), Future(16698 [BPH16]): Position({'amount': -28, 'last_sale_price': 1.466, 'cost_basis': 1.466, 'last_sale_date': Timestamp('2016-01-05 21:00:00+0000', tz='UTC'), 'asset': Future(16698 [BPH16])}), Future(16676 [KWH16]): Position({'amount': -33, 'last_sale_price': 460.0, 'cost_basis': 460.0, 'last_sale_date': Timestamp('2016-01-05 21:00:00+0000', tz='UTC'), 'asset': Future(16676 [KWH16])}), Future(16701 [WH16]): Position({'amount': -33, 'last_sale_price': 461.25, 'cost_basis': 461.25, 'last_sale_date': Timestamp('2016-01-05 21:00:00+0000', tz='UTC'), 'asset': Future(16701 [WH16])}), Future(16684 [CH16]): Position({'amount': -54, 'last_sale_price': 353.0, 'cost_basis': 353.0, 'last_sale_date': Timestamp('2016-01-05 21:00:00+0000', tz='UTC'), 'asset': Future(16684 [CH16])})}] 

In [11]:
import pandas as pd
df = pd.read_pickle('quick.pickle')

In [19]:
df.tail()

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2018-01-19 21:00:00+00:00,0.134987,0.070017,-0.177241,0.050776,0.064129,1.557199,99606.900,1.070017e+07,-3.084928e+08,0.0,...,-3.337968e+08,0.0,12,33.988795,1.060056e+07,-3.083842e+08,0.0,13,"[{'price': 93.031, 'sid': Future(25743 [I3H18]...",0.0
2018-01-22 21:00:00+00:00,0.134264,0.084399,-0.179532,0.059320,0.064265,1.585033,143824.100,1.084399e+07,-3.069634e+08,0.0,...,-3.343655e+08,0.0,12,39.199761,1.070017e+07,-3.084928e+08,0.0,14,"[{'price': 63.57, 'sid': Future(25506 [CLH18])...",0.0
2018-01-23 21:00:00+00:00,0.132010,0.083792,-0.261486,0.061568,0.062473,1.612821,-6075.100,1.083792e+07,-4.636086e+08,0.0,...,-4.912101e+08,0.0,13,37.535740,1.084399e+07,-3.069634e+08,0.0,15,"[{'price': 98.58, 'sid': Future(25582 [FFG18])...",0.0
2018-01-24 21:00:00+00:00,0.131524,0.098436,0.226900,0.061155,0.062815,1.347099,146435.500,1.098436e+07,-4.631705e+08,0.0,...,-4.911847e+08,0.0,13,42.394092,1.083792e+07,-4.636086e+08,0.0,16,"[{'price': 65.61, 'sid': Future(25506 [CLH18])...",0.0
2018-01-25 21:00:00+00:00,0.130599,0.096673,0.146619,0.061605,0.062133,1.386082,-17629.035,1.096673e+07,-4.610566e+08,0.0,...,-4.912213e+08,0.0,13,39.784051,1.098436e+07,-4.631705e+08,0.0,17,"[{'price': 1362.9, 'sid': Future(25515 [GCG18]...",0.0


In [20]:
transactions = df['transactions']
positions = df['positions']

In [21]:
transactions[1]

[{'amount': -377,
  'commission': None,
  'dt': Timestamp('2018-01-03 21:00:00+0000', tz='UTC'),
  'order_id': 'e6599b4e58c94cd0947086b106755955',
  'price': 98.59,
  'sid': Future(25376 [FFF18])},
 {'amount': 31,
  'commission': None,
  'dt': Timestamp('2018-01-03 21:00:00+0000', tz='UTC'),
  'order_id': '860bc2ce54ea48408f6b85439eea3cea',
  'price': 1.354,
  'sid': Future(25737 [BPH18])},
 {'amount': 10,
  'commission': None,
  'dt': Timestamp('2018-01-03 21:00:00+0000', tz='UTC'),
  'order_id': 'a5c830ddb2aa4a3a9cd5d990c666b4a7',
  'price': 1083.35,
  'sid': Future(25760 [PAH18])},
 {'amount': -144,
  'commission': None,
  'dt': Timestamp('2018-01-03 21:00:00+0000', tz='UTC'),
  'order_id': 'b7764e62e6d249fd89a16fdece216586',
  'price': 106.968,
  'sid': Future(25789 [TUH18])},
 {'amount': 13,
  'commission': None,
  'dt': Timestamp('2018-01-03 21:00:00+0000', tz='UTC'),
  'order_id': '922b946408624a56a77b044540571dac',
  'price': 1924.4,
  'sid': Future(25711 [MDH18])}]

In [22]:
positions[1]

[{'amount': -377,
  'cost_basis': 98.59,
  'last_sale_price': 98.59,
  'sid': Future(25376 [FFF18])},
 {'amount': 31,
  'cost_basis': 1.354,
  'last_sale_price': 1.354,
  'sid': Future(25737 [BPH18])},
 {'amount': 10,
  'cost_basis': 1083.35,
  'last_sale_price': 1083.35,
  'sid': Future(25760 [PAH18])},
 {'amount': -144,
  'cost_basis': 106.968,
  'last_sale_price': 106.968,
  'sid': Future(25789 [TUH18])},
 {'amount': 13,
  'cost_basis': 1924.4,
  'last_sale_price': 1924.4,
  'sid': Future(25711 [MDH18])}]